<a href="https://colab.research.google.com/github/atick-faisal/TAVI/blob/main/src/colab/MultiViewUNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Download Dataset

In [2]:
!gdown "1NWZPFHtwIOe3MbpuFtc7qJh3Q9AWrvSd"
!unzip -o "Dataset_v12_TAWSS.zip" > /dev/null


Downloading...
From: https://drive.google.com/uc?id=1NWZPFHtwIOe3MbpuFtc7qJh3Q9AWrvSd
To: /content/Dataset_v12_TAWSS.zip
100% 723M/723M [00:02<00:00, 297MB/s]


# Imports

In [21]:
!pip install git+https://github.com/tensorflow/examples.git
!pip install image-similarity-measures

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 4.1 MB/s 
     |████████████████████████████████| 19.3 MB 51.7 MB/s 
  Created wheel for phasepack: filename=phasepack-1.5-py3-none-any.whl size=26984 sha256=ce39cfd34e271c3d3dfb4537ad092e20023671aa754e37d838a6fadeebce0139
  Stored in directory: /root/.cache/pip/wheels/39/5c/b3/79292d7ac3f088e858805b5515b45cf5a365106397d936caf1
Successfully built phasepack


In [23]:
import os
import cv2
import random
import datetime
import matplotlib
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm
from numpy.typing import NDArray
from tensorflow_examples.models.pix2pix import pix2pix
from image_similarity_measures.quality_metrics import *

matplotlib.use('Agg')
plt.rcParams["font.size"] = 16


# Config

In [5]:
DATASET         = "v12"
MODEL_NAME      = "MobileUNet"
DATASET_PATH    = "/content/Images/"
TRAIN_DIR       = "Train/"
TEST_DIR        = "Val/"
INPUT_DIR       = "Input/"
TARGET_DIR      = "Target/"
MODEL_PATH      = "/content/drive/MyDrive/Research/TAVI/Models/"
PRED_PATH       = "/content/drive/MyDrive/Research/TAVI/Predictions/"

IMG_SIZE        = 128
BATCH_SIZE      = 8
BUFFER_SIZE     = 1000
VAL_SPLIT       = 0.2
LEARNING_RATE   = 0.001
N_EPOCHS        = 30
PATIENCE        = 30

EXP_NAME        = f"{MODEL_NAME}_I{IMG_SIZE}_B{BATCH_SIZE}_LR{LEARNING_RATE}_{DATASET}"


# Architecture

In [6]:
class UNet:
    def __init__(
        self,
        img_size: int,
        n_channels: int = 3,
        width: int = 32,
        depth: int = 4,
        kernel_size: int = 3
    ):
        self.img_size = img_size
        self.n_channels = n_channels
        self.width = width
        self.depth = depth
        self.kernel_size = kernel_size

    @staticmethod
    def conv(
        x: tf.Tensor,
        filters: int,
        kernel_size: int
    ) -> tf.Tensor:
        for i in range(2):
            x = tf.keras.layers.Conv2D(
                filters=filters,
                kernel_size=kernel_size,
                strides=1,
                padding="same",
                data_format="channels_last",
                dilation_rate=1,
                groups=1,
                activation=None,
                use_bias=True,
                kernel_initializer="glorot_uniform",
                bias_initializer="zeros"
            )(x)

            x = tf.keras.layers.BatchNormalization()(x)
            x = tf.keras.layers.Activation("relu")(x)

        return x

    @staticmethod
    def deconv(x: tf.Tensor, filters: int) -> tf.Tensor:
        x = tf.keras.layers.Conv2DTranspose(
            filters=filters,
            kernel_size=2,
            strides=2,
            padding="same",
            output_padding=None,
            data_format=None,
            dilation_rate=1,
            activation=None,
            use_bias=True,
            kernel_initializer="glorot_uniform",
            bias_initializer="zeros"
        )(x)

        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation("relu")(x)

        return x

    @staticmethod
    def output(x: tf.Tensor) -> tf.Tensor:
        return tf.keras.layers.Conv2D(3, (1, 1), activation="sigmoid")(x)

    @staticmethod
    def pool(x: tf.Tensor, pool_size: int = 2) -> tf.Tensor:
        return tf.keras.layers.MaxPool2D(pool_size)(x)

    def __call__(self) -> tf.keras.Model:
        inputs = tf.keras.layers.Input(
            shape=(self.img_size, self.img_size, self.n_channels)
        )

        # scaled = tf.keras.layers.Rescaling(1./255.0, offset=0)(inputs)

        # ------------------ Downsampling ---------------------
        downsample_layers = []
        downsample_layers.append(
            self.conv(
                x=inputs,
                filters=self.width,
                kernel_size=self.kernel_size
            )
        )
        for i in range(1, self.depth):
            filters = int((2 ** i) * self.width)
            downsample_layers.append(
                self.pool(
                    self.conv(
                        x=downsample_layers[i - 1],
                        filters=filters,
                        kernel_size=self.kernel_size
                    )
                )
            )

        # ------------------- Features --------------------
        n_features = int((2 ** self.depth) * self.width)
        self.features = self.pool(
            self.conv(
                x=downsample_layers[-1],
                filters=n_features,
                kernel_size=self.kernel_size
            )
        )

        # ------------------- Upsampling --------------------
        upsample_layers = []
        upsample_layers.append(self.features)
        for i in range(1, self.depth + 1):
            filters = int((2 ** (self.depth - i)) * self.width)
            upsample_layers.append(
                self.conv(
                    x=tf.keras.layers.concatenate([
                        downsample_layers[self.depth - i],
                        self.deconv(
                            x=upsample_layers[i - 1],
                            filters=filters
                        )
                    ]),
                    filters=filters,
                    kernel_size=self.kernel_size
                )
            )

        # ---------------------- Output -----------------------
        outputs = self.output(upsample_layers[-1])

        return tf.keras.Model(inputs, outputs)


In [7]:
class MobileUNet:
    def __init__(
        self,
        img_size: int,
    ):
        self.img_shape = (img_size, img_size, 3)

        self.base_model = tf.keras.applications.MobileNetV2(
            input_shape=self.img_shape, 
            include_top=False
        )

        # Use the activations of these layers
        self.layer_names = [
            'block_1_expand_relu',   # 64x64
            'block_3_expand_relu',   # 32x32
            'block_6_expand_relu',   # 16x16
            'block_13_expand_relu',  # 8x8
            'block_16_project',      # 4x4
        ]
        self.base_model_outputs = [self.base_model.get_layer(name).output for name in self.layer_names]

        # Create the feature extraction model
        self.down_stack = tf.keras.Model(inputs=self.base_model.input, outputs=self.base_model_outputs)

        self.down_stack.trainable = True

        self.up_stack = [
            pix2pix.upsample(512, 3),  # 4x4 -> 8x8
            pix2pix.upsample(256, 3),  # 8x8 -> 16x16
            pix2pix.upsample(128, 3),  # 16x16 -> 32x32
            pix2pix.upsample(64, 3),   # 32x32 -> 64x64
            pix2pix.upsample(32, 3),   # 32x32 -> 64x64
        ]

    def __call__(self):
        inputs = tf.keras.layers.Input(shape=self.img_shape)

        # Downsampling through the model
        skips = self.down_stack(inputs)
        x = skips[-1]
        skips = reversed(skips[:-1])

        # Upsampling and establishing the skip connections
        for up, skip in zip(self.up_stack, skips):
            x = up(x)
            concat = tf.keras.layers.Concatenate()
            x = concat([x, skip])

        # This is the last layer of the model
        x = tf.keras.layers.Conv2DTranspose(
            filters=16, kernel_size=3, strides=2,
            padding='same')(x)

        x = tf.keras.layers.Conv2D(3, (1, 1), activation="sigmoid")(x)

        # x = last(x)

        return tf.keras.Model(inputs=inputs, outputs=x)

    

# Loss Functions

In [8]:
def attention_mse(y_true, y_pred):
    _y_true = y_true[y_true != 1.0]
    _y_pred = y_pred[y_true != 1.0]
    squared_difference = tf.square(_y_true - _y_pred)
    return tf.reduce_mean(squared_difference, axis=-1)


def attention_mae(y_true, y_pred):
    _y_true = y_true[y_true != 1.0]
    _y_pred = y_pred[y_true != 1.0]
    squared_difference = tf.abs(_y_true - _y_pred)
    return tf.reduce_mean(squared_difference, axis=-1)


# Utils

In [9]:
def load_data_from_dir(path: str) -> tf.data.Dataset:
    return tf.keras.utils.image_dataset_from_directory(
        directory=path,
        labels=None,
        color_mode='rgb',
        batch_size=BATCH_SIZE,
        image_size=(IMG_SIZE, IMG_SIZE),
        shuffle=True,
        seed=42,
        # validation_split=VAL_SPLIT,
        # subset=subset,
        interpolation='bilinear',
        follow_links=False,
        crop_to_aspect_ratio=False
    )


# Dataloaders

In [10]:
trainX = load_data_from_dir(os.path.join(DATASET_PATH, TRAIN_DIR, INPUT_DIR))
trainY = load_data_from_dir(os.path.join(DATASET_PATH, TRAIN_DIR, TARGET_DIR))
testX = load_data_from_dir(os.path.join(DATASET_PATH, TEST_DIR, INPUT_DIR))
testY = load_data_from_dir(os.path.join(DATASET_PATH, TEST_DIR, TARGET_DIR))

train_ds = tf.data.Dataset.zip((trainX, trainY))
test_ds = tf.data.Dataset.zip((testX, testY))

print(train_ds.element_spec)
print(test_ds.element_spec)


Found 6624 files belonging to 1 classes.
Found 6624 files belonging to 1 classes.
Found 552 files belonging to 1 classes.
Found 552 files belonging to 1 classes.
(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None))
(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None))


# Normalization

In [11]:
# def normalize(input_image, output_image):
#     input_image = tf.cast(input_image, tf.float32) / 255.0
#     output_image = tf.cast(input_image, tf.float32) / 255.0
#     return input_image, output_image

In [12]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), normalization_layer(y)))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), normalization_layer(y)))

# Augmentation

In [13]:
class Augment(tf.keras.layers.Layer):
    def __init__(self, seed=42):
        super().__init__()
        # both use the same seed, so they'll make the same random changes.
        self.augment_inputs = tf.keras.layers.RandomFlip(mode="horizontal", seed=seed)
        self.augment_labels = tf.keras.layers.RandomFlip(mode="horizontal", seed=seed)

    def call(self, inputs, labels):
        inputs = self.augment_inputs(inputs)
        labels = self.augment_labels(labels)
        return inputs, labels

# Optimization

In [14]:
AUTOTUNE = tf.data.AUTOTUNE

train_batches = (
    train_ds
    .cache()
    # .shuffle(BUFFER_SIZE)
    # .batch(BATCH_SIZE)
    # .repeat()
    .map(Augment())
    .prefetch(buffer_size=AUTOTUNE)
)

test_batches = (test_ds)


# Training Config

In [15]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=PATIENCE,
        restore_best_weights=True
    )
]

optimizer = tf.keras.optimizers.Adam(
    learning_rate=LEARNING_RATE
)

# model = UNet(IMG_SIZE)()
model = MobileUNet(IMG_SIZE)()

model.compile(
    loss=attention_mse,
    optimizer=optimizer,
    metrics=['mean_squared_error']
)


# Training

In [16]:
history = model.fit(
    train_batches,
    validation_data=test_batches,
    epochs=N_EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=callbacks,
    verbose=1
)


Epoch 1/30
828/828 [==============================] - 63s 66ms/step - loss: 0.0253 - mean_squared_error: 0.3941 - val_loss: 0.0475 - val_mean_squared_error: 0.3453
Epoch 2/30
828/828 [==============================] - 42s 51ms/step - loss: 0.0180 - mean_squared_error: 0.3493 - val_loss: 0.0300 - val_mean_squared_error: 0.2739
Epoch 3/30
828/828 [==============================] - 42s 51ms/step - loss: 0.0135 - mean_squared_error: 0.2916 - val_loss: 0.0204 - val_mean_squared_error: 0.1985
Epoch 4/30
828/828 [==============================] - 42s 50ms/step - loss: 0.0102 - mean_squared_error: 0.2150 - val_loss: 0.0131 - val_mean_squared_error: 0.1120
Epoch 5/30
828/828 [==============================] - 42s 51ms/step - loss: 0.0095 - mean_squared_error: 0.1890 - val_loss: 0.0108 - val_mean_squared_error: 0.0544
Epoch 6/30
828/828 [==============================] - 42s 50ms/step - loss: 0.0089 - mean_squared_error: 0.1607 - val_loss: 0.0115 - val_mean_squared_error: 0.0345
Epoch 7/30
828/8

# Save Model

In [17]:
timestamp = datetime.datetime.now().strftime('%b-%d-%I:%M%p')
model_path = os.path.join(MODEL_PATH, EXP_NAME)
if not os.path.exists(model_path):
    os.makedirs(model_path)

model.save(os.path.join(model_path, timestamp))

# Save Predictions

In [28]:
test_ds_unbatched = test_batches.unbatch()

pred_path = os.path.join(PRED_PATH, EXP_NAME, timestamp)
if not os.path.exists(pred_path):
    os.makedirs(pred_path)

metrics = pd.DataFrame()

for idx, (input, target) in enumerate(tqdm(test_ds_unbatched)):

    target = target.numpy()
    prediction = tf.squeeze(
        model.predict(
            tf.expand_dims(input, axis=0)
            )
        ).numpy()
    prediction[target == 1.0] = 1.0

    _metrics = pd.DataFrame([[
            rmse(target, prediction),
            psnr(target, prediction),
            ssim(target, prediction),
            fsim(target, prediction),
            issm(target, prediction),
            sre(target, prediction),
            sam(target, prediction),
            uiq(target, prediction),
        ]],
        columns=["RMSE", "PSNR", "SSIM", "FSIM", "ISSM", "SRE", "SAM", "UIQ"]
    )

    metrics = pd.concat([metrics, _metrics], axis=0)

    if not idx % 40 == 0:
        continue

    plt.figure(figsize=(12, 4))
    plt.subplot(1, 3, 1)
    plt.imshow(input)
    plt.axis("off")
    plt.title("INPUT")
    plt.subplot(1, 3, 2)
    plt.imshow(target)
    plt.axis("off")
    plt.title("TARGET")
    plt.subplot(1, 3, 3)
    plt.imshow(prediction)
    plt.axis("off")
    plt.title("PREDICTION")
    plt.tight_layout()
    plt.savefig(os.path.join(pred_path, f"{idx}.png"))
    plt.close()  


0it [00:00, ?it/s]

1/1 [==============================] - 0s 47ms/step


1it [00:05,  5.31s/it]

1/1 [==============================] - 0s 23ms/step


2it [00:09,  4.89s/it]

...

550it [42:31,  4.64s/it]

1/1 [==============================] - 0s 21ms/step


551it [42:36,  4.70s/it]

1/1 [==============================] - 0s 20ms/step


552it [42:41,  4.64s/it]


In [19]:
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 6))
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Attention MAE')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.tight_layout()
plt.savefig(os.path.join(model_path, timestamp + ".png"))
plt.close()

In [20]:
history

In [29]:
metrics

,RMSE,PSNR,SSIM,FSIM,ISSM,SRE,SAM,UIQ
0,0.000005,106.330986,1.0,0.986872,0.0,40.349758,0.393067,0.864831
0,0.000006,104.309990,1.0,0.990317,0.0,40.868201,0.616709,0.848967
0,0.000008,101.961196,1.0,0.979266,0.0,40.913385,0.784501,0.865688
0,0.000005,106.366634,1.0,0.984267,0.0,38.712033,0.349486,0.917367
0,0.000007,102.680523,1.0,0.974752,0.0,38.107613,0.480382,0.889561
...,...,...,...,...,...,...,...,...
0,0.000005,106.801753,1.0,0.990090,0.0,40.926667,0.372600,0.892624
0,0.000005,105.848274,1.0,0.984536,0.0,41.105306,0.470067,0.876452
0,0.000006,104.830704,1.0,0.980286,0.0,39.831557,0.458005,0.843419
0,0.000006,104.464157,1.0,0.986190,0.0,38.832027,0.454618,0.855725
